In [22]:
import base64
import json
from flask import Flask, jsonify, request
import numpy as np
import cv2

In [58]:
app = Flask(__name__)

@app.route('/api/image', methods=['GET'])
def send_image():
    # 이미지 파일 읽어오기
    with open("C:/Users/do150/Desktop/main_img_1158px_01.png", "rb") as image_file:
        image_bytes = image_file.read()

    # Base64 인코딩
    base64_image = base64.b64encode(image_bytes).decode('utf-8')

    # JSON 데이터 만들기
    response_data = {'imageData': base64_image}
    response = jsonify(response_data)

    return response

@app.route('/image', methods=['POST'])
def upload_image():
    image = request.get_json()
    img = image['image']
    imgdata = base64.b64decode(img)
    encoded_img = np.fromstring(imgdata, dtype = np.uint8)
    img_cv = cv2.imdecode(encoded_img, cv2.IMREAD_COLOR)
    cv2.imwrite('C:/Users/do150/Desktop/upload/image.jpg', img_cv)
    return 'Image uploaded successfully.'

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=83)


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.1.38:83/ (Press CTRL+C to quit)
C:\Users\do150\AppData\Local\Temp\ipykernel_6532\1609159443.py:23: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  encoded_img = np.fromstring(imgdata, dtype = np.uint8)
192.168.1.38 - - [14/Apr/2023 19:46:24] "POST /image HTTP/1.1" 200 -


(640, 1152, 3)


In [39]:
from PIL import Image

# 입력 파일 경로와 출력 파일 경로 설정
input_file = "C:/Users/do150/Desktop/de/out-0.png"
output_file = "C:/Users/do150/Desktop/de/out-1.jpg"

# png 이미지 열기
with Image.open(input_file) as img:
    # 이미지를 RGB 모드로 변환하고 jpg로 저장
    img.convert("RGB").save(output_file)

In [49]:
image_name = 'C:/Users/do150/Desktop/de/out-1.jpg'
img = cv2.imread(image_name)
jpg_img = cv2.imencode('.jpg', img)
b64_string = base64.b64encode(jpg_img[1]).decode('utf-8')

imgdata = base64.b64decode(b64_string)
encoded_img = np.fromstring(imgdata, dtype = np.uint8)
img_cv = cv2.imdecode(encoded_img, cv2.IMREAD_COLOR)
cv2.imshow('image', img_cv)
cv2.waitKey(0)
cv2.destroyAllWindows()

C:\Users\do150\AppData\Local\Temp\ipykernel_6532\1988471089.py:7: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  encoded_img = np.fromstring(imgdata, dtype = np.uint8)
